In [8]:
import ROOT as rt
# import root_numpy as rtnp
import csv
import re
import sys
import collections

from collections import OrderedDict
import uproot
import pandas as pd
import os
import scipy
import awkward
import numpy as np
import time
import numba
from numba import jit
from matplotlib import pyplot as plt
sys.path.append(os.getenv('HOME')+'/gpu/christiw/llp/delayed_jet_analyzer/lib/')
from histo_utilities import create_TH1D, create_TH2D, std_color_list, create_TGraph, make_ratio_plot

import CMS_lumi, tdrstyle
a = tdrstyle.setTDRStyle()
CMS_lumi.writeExtraText = 0

print(sys.version)

3.6.8 (default, Aug  7 2019, 17:28:10) 
[GCC 4.8.5 20150623 (Red Hat 4.8.5-39)]


# Load ntuples

In [12]:
fpath_bkg =OrderedDict()
tree_bkg = OrderedDict()
tree_sig = OrderedDict()
fpath_sig =OrderedDict()


start_t = time.time()

path = '/storage/af/group/phys_exotica/delayedjets/HNL/skim/'


isData = 0
isSignal = 1
category = 'muon'
if isData:
    fpath_bkg['data'] = path+ 'EGamma_2018A.root'
    fpath_bkg['data'] = '/storage/af/group/phys_exotica/delayedjets/HNL/displacedJetMuonAnalyzer/V1p17/Data2018/v5/v2/normalized/2018B.root'
else:
    if isSignal:
        sample = ['HNL_'+category+'Type_mHNL10p0_pl10000',
        'HNL_'+category+'Type_mHNL10p0_pl1000',
        'HNL_'+category+'Type_mHNL10p0_pl100',
        'HNL_'+category+'Type_mHNL1p0_pl1000',
        'HNL_'+category+'Type_mHNL1p0_pl100',
        'HNL_'+category+'Type_mHNL1p0_pl10',
        'HNL_'+category+'Type_mHNL2p0_pl1000',
        'HNL_'+category+'Type_mHNL2p0_pl100',
        'HNL_'+category+'Type_mHNL2p0_pl10',
        'HNL_'+category+'Type_mHNL4p0_pl1000',
        'HNL_'+category+'Type_mHNL4p0_pl100',
        'HNL_'+category+'Type_mHNL4p0_pl10',
        'HNL_'+category+'Type_mHNL7p0_pl10000',
        'HNL_'+category+'Type_mHNL7p0_pl1000',
        'HNL_electronType_mHNL7p0_pl100',
            ]
    else: sample = ['WJetsToLNu']

    for s in sample:
        fpath_bkg[s] = path + s + '_1pb_weighted.root'
NEvents = {}
Total = {}
accep = {}
acc_met = {}
for k,v in fpath_bkg.items():
    print (k, v)
    root_dir = uproot.open(v) 
    tree_bkg[k] = root_dir['MuonSystem']
    NEvents[k] = root_dir['NEvents'][1]
    a = tree_bkg[k]["weight"].array()
    print("NEvents",NEvents[k],np.sum(a))


HNL_muonType_mHNL10p0_pl10000 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_muonType_mHNL10p0_pl10000_1pb_weighted.root
NEvents 8.060411 1.210004e-05
HNL_muonType_mHNL10p0_pl1000 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_muonType_mHNL10p0_pl1000_1pb_weighted.root
NEvents 80.781876 0.00012099969
HNL_muonType_mHNL10p0_pl100 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_muonType_mHNL10p0_pl100_1pb_weighted.root
NEvents 808.63214 0.0012099985
HNL_muonType_mHNL1p0_pl1000 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_muonType_mHNL1p0_pl1000_1pb_weighted.root
NEvents 9061672.0 13.569931
HNL_muonType_mHNL1p0_pl100 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_muonType_mHNL1p0_pl100_1pb_weighted.root
NEvents 90733710.0 135.69987
HNL_muonType_mHNL1p0_pl10 /storage/af/group/phys_exotica/delayedjets/HNL/skim/HNL_muonType_mHNL1p0_pl10_1pb_weighted.root
NEvents 905525060.0 1356.997
HNL_muonType_mHNL2p0_pl1000 /storage/af/group/phys_exotica/delaye

# load Wpt weights

In [10]:
path =  os.getenv('HOME')+'/login-1/christiw/LLP/CMSSW_9_4_4/src/llp_analyzer/data/WPtWeights/'
target = uproot.open(path + 'wp-13tev-cms.root')['s_qt']
source = uproot.open(path + 'WPT.root')['Wpt']
  


# background(WJetsMC/data) cut flow

In [11]:
JET_PT_CUT = 20.0
MUON_PT_CUT = 10.0
N_RECHIT_CUT = 100
jetPt_cut = 30

weight = {}
sel_ev = {}

ncluster = 1
cluster_index = '3'
table = OrderedDict()
table['acc'] = "Acceptance "


table['trigger'] = "single lepton trigger"
table['lep'] = "$N_{lepton} > 0$ "
table['met'] = "MET cut "
table['met_filter'] = "MET filters "


table['1_cluster'] = "$N_{cluster} \geq 1$ "
table['jet_veto'] = "jet veto "
table['muon_veto'] = "muon veto "
table['mb1'] = "MB1 veto "
table['rpc'] = "rpc matching "
table['mb1_adjacent'] = "mb1_adjacent "
table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
table['nrechits'] = r"$N_{rechits}$ cut "
table['mb2'] = 'mb2'
table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
table['nrechits'] = r"$N_{rechits}$ cut "

for k, T in tree_bkg.items():
    if k == 'data':
#         weight = T.array('weight')*0.0+137000/939.0
        weight = T.array('weight')*0.0+1

    elif k == 'WJetsToLNu':
        gWPt[k] = T.array('gWPt')
        weight = T.array('weight')*T.array('pileupWeight')*137000
        weight *= target.values[np.argmax(target.edges>gWPt[k][:,None],axis=1)-1]/11178549.542718 # integral
        weight /= source.values[np.argmax(source.edges>gWPt[k][:,None],axis=1)-1]
    else:continue
    ########### SELECTION: ELECTRON ############

    sel_ele = np.logical_and(T.array('lepPt') > 35, np.abs(T.array('lepEta'))<2.4)
    sel_ele = np.logical_and(sel_ele, np.abs(T.array('lepPdgId')) == 11)
    sel_ele = np.logical_and(sel_ele, np.abs(T.array('lepPassId')) == 1)


    ########### SELECTION: EVENTS ############


    total = NEvents[k]
    acc_met = np.sum(weight)
    sel_temp = np.sum(weight)

    sel_ev[k] =  T.array('SingleEleTrigger')
#     print("trigger", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["trigger"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]]))
    sel_temp = np.sum(weight[sel_ev[k]])

    
    sel_ev[k] = np.logical_and(sel_ev[k] ,sel_ele.sum()==1)
#     print("single lepton", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["lep"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]]))
    sel_temp = np.sum(weight[sel_ev[k]])

    
    
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise')>=30)
#     print("MET", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["met"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]]))
    sel_temp = np.sum(weight[sel_ev[k]])

    
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('Flag2_all') == 1)
#     print("MET filters", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["met_filter"] += " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]]))
    sel_temp = np.sum(weight[sel_ev[k]])






    ########### SELECTION: CLUSTERS ############

    
    
    binD = np.logical_and(T.array('dtRechitClusterSize')>=100, np.abs(T.array('dtRechitClusterMetEENoise_dPhi'))<1)

    
    sel_rechitcluster = np.abs(T.array('dtRechitClusterSize'))>=0
    
    #signal region blind bin D
    if k == 'data': sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_not(binD))
    

    
#     print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["1_cluster"] +=  " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))  

    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

    

    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterJetVetoPt') < JET_PT_CUT)
#     print("jet cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["jet_veto"] +=   " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))  
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
    
    
    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterMuonVetoPt') < MUON_PT_CUT)
#     print("muon cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))  
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_0p5') <= 1)
#     print("mb1", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["mb1"] +=  " & {0:6.2f} & {1:6.5f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))  
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_RPChits_dPhi0p5') > 0)
#     print("rpc matching", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["rpc"] +=  " & {0:6.2f} & {1:6.5f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))  
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_minus') <= 8)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_plus') <= 8)

#     print("mb1 adjacent", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["mb1_adjacent"] +=  " & {0:6.2f} & {1:6.5f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))  
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

    
    
    
    
    
    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 1)
#     print("dphi(cluster, met) cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["dphi"] +=   " & {0:6.2f} & {1:6.5f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))  
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterSize')>=100)

#     print("N rechits", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["nrechits"] +=  " & {0:6.2f} & {1:6.5f} & {2:6.0f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)]))  
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



for k,v in table.items():
    print(v+r" \\")



Acceptance  \\
single lepton trigger &  44.87 &  44.87 & 3151008 \\
$N_{lepton} > 0$  &  33.05 &  14.83 & 1041387 \\
MET cut  &  69.34 &  10.28 & 722079 \\
MET filters  &  99.95 &  10.28 & 721746 \\
$N_{cluster} \geq 1$  &  14.47 &   1.49 & 104442 \\
jet veto  &  62.73 &   0.93 &  65516 \\
muon veto  &  84.54 &   0.79 &  55386 \\
MB1 veto  &   2.94 & 0.02323 &   1631 \\
rpc matching  &  72.47 & 0.01683 &   1182 \\
mb1_adjacent  &  91.29 & 0.01537 &   1079 \\
$\Delta\phi\mathrm{ (cluster,MET)}$  &  32.44 & 0.00498 &    350 \\
$N_{rechits}$ cut  &   0.00 & 0.00000 &      0 \\
mb2 \\


# background n-1 cut efficiency

In [6]:
JET_PT_CUT = 20.0
MUON_PT_CUT = 10.0
N_RECHIT_CUT = 100
jetPt_cut = 30

weight = {}
sel_ev = {}

ncluster = 1
cluster_index = '3'
table = OrderedDict()
table['acc'] = "Acceptance "


table['trigger'] = "single lepton trigger"
table['lep'] = "$N_{lepton} > 0$ "
table['met'] = "MET cut "
table['met_filter'] = "MET filters "


table['1_cluster'] = "$N_{cluster} \geq 1$ "
table['jet_veto'] = "jet veto "
table['muon_veto'] = "muon veto "
table['mb1'] = "MB1 veto "
table['rpc'] = "rpc matching "
table['mb1_adjacent'] = "mb1_adjacent "
table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
table['nrechits'] = r"$N_{rechits}$ cut "
table['mb2'] = 'mb2'
table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
table['nrechits'] = r"$N_{rechits}$ cut "

for k, T in tree_bkg.items():
    if k == 'data':
#         weight = T.array('weight')*0.0+137000/939.0
        weight = T.array('weight')*0.0+1

    elif k == 'WJetsToLNu':
        gWPt[k] = T.array('gWPt')
        weight = T.array('weight')*T.array('pileupWeight')*137000
        weight *= target.values[np.argmax(target.edges>gWPt[k][:,None],axis=1)-1]/11178549.542718 # integral
        weight /= source.values[np.argmax(source.edges>gWPt[k][:,None],axis=1)-1]
    else:continue
    ########### SELECTION: ELECTRON ############

    sel_ele = np.logical_and(T.array('lepPt') > 35, np.abs(T.array('lepEta'))<2.4)
    sel_ele = np.logical_and(sel_ele, np.abs(T.array('lepPdgId')) == 11)
    sel_ele = np.logical_and(sel_ele, np.abs(T.array('lepPassId')) == 1)


    ########### SELECTION: EVENTS ############


    total = NEvents[k]
    acc_met = np.sum(weight)
    sel_temp = np.sum(weight)

    sel_ev[k] =  T.array('SingleEleTrigger')
    sel_ev[k] = np.logical_and(sel_ev[k] ,sel_ele.sum()==1)
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise')>=30)
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('Flag2_all') == 1)
  



    ########### SELECTION: CLUSTERS ############

    
    
    binD = np.logical_and(T.array('dtRechitClusterSize')>=100, np.abs(T.array('dtRechitClusterMetEENoise_dPhi'))<1)

    
    sel_rechitcluster = np.abs(T.array('dtRechitClusterSize'))>=0
    #signal region blind D
    if k == 'data': sel_rechitcluster = np.logical_and(sel_rechitcluster, np.logical_not(binD))

    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

    

    
    vetos = [T.array('dtRechitClusterJetVetoPt') < JET_PT_CUT, \
             T.array('dtRechitClusterMuonVetoPt') < MUON_PT_CUT, \
             T.array('dtRechitCluster_match_MB1hits_0p5') <= 1, \
             T.array('dtRechitCluster_match_RPChits_dPhi0p5') > 0, \
             np.logical_and(T.array('dtRechitCluster_match_MB1hits_cosmics_minus') <= 8, T.array('dtRechitCluster_match_MB1hits_cosmics_plus') <= 8)]
    
    for i, v in enumerate(vetos):
        sel_rechitcluster_temp = sel_rechitcluster.copy()
        for j, w in enumerate(vetos):
            if i == j:continue
            sel_rechitcluster_temp = np.logical_and( sel_rechitcluster_temp, w)
        sel_n_1 = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster_temp.sum()>=1)])
        sel_rechitcluster_temp = np.logical_and( sel_rechitcluster_temp, v)
        print(i, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster_temp.sum()>=1)])/sel_n_1)
    

    
    

trigger 0.44874316 0.44874316 3151008
single lepton 0.1483066 0.3304933 1041387
MET 0.10283313 0.693382 722079
MET filters 0.10278571 0.99953884 721746
cluster efficiency 0.014873854 0.14470741
0 0.88442624
1 0.81866467
2 0.02902254
3 0.7280702
4 0.91285956


# signal cut flow

In [13]:
JET_PT_CUT = 10.0
MUON_PT_CUT = 20.0
N_RECHIT_CUT = 90
jetPt_cut = 50
tightid = False


intime = True
DPHI_CUT = 1
weight = {}
weight_event = {}
weight_ctau = {}
lumiSec = {}
evtNum = {}
ndt = {}
npv = {}
npu = {}
runNum = {}
ndt_JetMuonVetoCluster0p4_Me1112Veto = {}
gLLP_dt = {}
dtClusterSize = {}
dtClusterTime = {}
ndtClusters = {}
selections_cluster = {}
sel_cluster = {}
sel_jetveto = {}
met_trigger = {}
met = {}
gLLP_beta = {}
jetPt = {}
jetPhi = {}
metPhi = {}
angle ={}
nLeptons = {}
genJetPt = {}
genJetPhi = {}
genMet = {}
genMetPhi = {}
pileupWeight = {}
gLLP_ctau = {}
npv = {}
nRechitClusters = {}
nJets = {}
nJets_50gev = {}
dtRechitClusterTimeDiff = {}
dtRechitCluster_match_gLLP = {}
dtRechitClusterXYSpread = {}
dtRechitClusterXSpread = {}
dtRechitClusterYSpread = {}
dtRechitClusterNStation = {}
dtRechitClusterEtaPhiSpread = {}
dtRechitClusterPhiSpread = {}
dtRechitClusterEtaSpread = {}
dtRechitClusterNStation10 = {}
dtRechitClusterAvgStation10 = {}
dtRechitClusterX = {}
dtRechitClusterY = {}
dtRechitClusterZ = {}
dtRechitClusterPhi = {}
dtClusterJetVetoPt = {}
dtRechitClusterEta = {}
dtRechitCluster_match_gLLP_index = {}
dtRechitClusterMaxStationRatio = {}
dtRechitClusterNStation = {}
dtRechitClusterNChamber = {}
dtRechitClusterAvgStation = {}
dtRechitClusterMet_dPhi = {}
dtRechitCluster2_match_Me1112_0p8 = {}
jetMet_dPhiMin30 = {}
jetMet_dPhiMin = {}
dphiMet_cluster = {}
nRechits_sr = {}
jetMet_dPhiMin30_sr = {}
gLLP_decay_vertex_z = {}
gLLP_decay_vertex_x = {}
gLLP_decay_vertex_y = {}
gLLP_decay_vertex_r = {}
ctau = {}
bdt_score = {}
a = {}
b = {}
c = {}
d = {}
sel_ev = {}
bdt_sel = {}
gHiggsPt = {}
gLLP_pt = {}
gLLP2_pt = {}
gLLP2_decay_vertex_r = {}
gLLP2_decay_vertex_z = {}
dtRechitCluster2_match_gLLP_decay_z = {}
dtRechitCluster2_match_gLLP_decay_r = {}
gParticleId = {}
gParticlePt = {}
dtRechitCluster_match_gLLP_decay_r = {}
dtRechitCluster_match_gLLP_decay_z = {}
trigger_bits = {}
gWPt = {}
# keys = ['mc_sr']
dphi = {}
ncluster = 1
cluster_index = '3'
table = OrderedDict()
table['acc'] = "Acceptance "


table['trigger'] = "single lepton trigger"
table['lep'] = "$N_{lepton} > 0$ "
table['met'] = "MET cut "
table['met_filter'] = "MET filters "


table['1_cluster'] = "$N_{cluster} \geq 1$ "
table['jet_veto'] = "jet veto "
table['muon_veto'] = "muon veto "
table['mb1'] = "MB1 veto "
table['rpc'] = "rpc matching "
table['mb1_adjacent'] = "mb1_adjacent "
table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
table['nrechits'] = r"$N_{rechits}$ cut "
table['mb2'] = 'mb2'
table['dphi'] = r"$\Delta\phi\mathrm{ (cluster,MET)}$ "
table['nrechits'] = r"$N_{rechits}$ cut "

for k, T in tree_bkg.items():
    print(k)
    weight = T.array('weight')*T.array('pileupWeight')
    gWPt[k] = T.array('gWPt')
    if 'HNL' in k:
        weight *= target.values[np.argmax(target.edges>gWPt[k][:,None],axis=1)-1]/11178549.542718 # integral
        weight /= source.values[np.argmax(source.edges>gWPt[k][:,None],axis=1)-1]
    
    ########### SELECTION: ELECTRON ############

    sel_ele = np.logical_and(T.array('lepPt') > 35, np.abs(T.array('lepEta'))<2.4)
    sel_ele = np.logical_and(sel_ele, np.abs(T.array('lepPdgId')) == 11)
    sel_ele = np.logical_and(sel_ele, np.abs(T.array('lepPassId')) == 1)
    
    sel_muon = np.logical_and(T.array('lepPt') > 25, np.abs(T.array('lepEta'))<2.4)
    sel_muon = np.logical_and(sel_muon, np.abs(T.array('lepPdgId')) == 13)
    sel_muon = np.logical_and(sel_muon, np.abs(T.array('lepPassId')) == 1)
    
    
    ########### SELECTION: EVENTS ############

    sel_ev[k] = np.sum(T.array('gLLP_dt'),axis = 1) >= 1
    acceptance = np.sum(weight[sel_ev[k]])
    acc_met = np.sum(weight[sel_ev[k]])

    total = np.sum(weight)
#     print("acceptance", acceptance/total, np.count_nonzero(sel_ev[k]))
    table["acc"] += " & {0:6.3f} & {1:6.3f}".format(100*acceptance/total,100*acceptance/total)
    sel_temp = np.sum(weight[sel_ev[k]])

    

    if category == 'electron': sel_ev[k] = np.logical_and(sel_ev[k], T.array('SingleEleTrigger'))
    else: sel_ev[k] = np.logical_and(sel_ev[k], T.array('SingleMuonTrigger'))
#     print("trigger", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["trigger"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)
    sel_temp = np.sum(weight[sel_ev[k]])

    
    if category == 'electron': sel_ev[k] = np.logical_and(sel_ev[k] ,sel_ele.sum()==1)
    else: sel_ev[k] = np.logical_and(sel_ev[k] ,sel_muon.sum()==1)
#     print("single lepton", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["lep"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)
    sel_temp = np.sum(weight[sel_ev[k]])

    
    
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('metEENoise')>=30)
#     print("MET", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["met"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)
    sel_temp = np.sum(weight[sel_ev[k]])

    
    sel_ev[k] = np.logical_and(sel_ev[k] ,T.array('Flag2_all') == 1)
#     print("MET filters", np.sum(weight[sel_ev[k]])/acc_met, np.sum(weight[sel_ev[k]])/sel_temp,np.count_nonzero(sel_ev[k]))
    table["met_filter"] += " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[sel_ev[k]])/sel_temp, 100*np.sum(weight[sel_ev[k]])/acc_met)
    sel_temp = np.sum(weight[sel_ev[k]])





    ########### SELECTION: CLUSTERS ############

    sel_rechitcluster = T.array('dtRechitCluster_match_gLLP_dt')
#     print("cluster efficiency", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["1_cluster"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)  

    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])




    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterJetVetoPt') < 20)
#     print("jet cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["jet_veto"] +=   " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)       
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])
    
    
    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterMuonVetoPt') < 10)
#     print("muon cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["muon_veto"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)   
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_0p5') <= 1)
#     print("mb1", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["mb1"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)   
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])



    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_RPChits_dPhi0p5') > 0)
#     print("rpc matching", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["rpc"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)   
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_minus') <= 8)
    sel_rechitcluster = np.logical_and(sel_rechitcluster, T.array('dtRechitCluster_match_MB1hits_cosmics_plus') <= 8)

#     print("mb1 adjacent", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["mb1_adjacent"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)     
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

    sel_rechitcluster = np.logical_and(sel_rechitcluster, np.abs(T.array('dtRechitClusterMetEENoise_dPhi')) < 1)
#     print("dphi(cluster, met) cut", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["dphi"] +=   " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)     
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])


    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterSize')>=100)

#     print("N rechits", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["nrechits"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)        
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

    sel_rechitcluster = np.logical_and( sel_rechitcluster, T.array('dtRechitClusterMaxStation')==2)

#     print("MB2", np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met, np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp)
    table["mb2"] +=  " & {0:6.2f} & {1:6.2f}".format(100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/sel_temp,\
                                                      100*np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])/acc_met)        
    sel_temp = np.sum(weight[np.logical_and(sel_ev[k], sel_rechitcluster.sum()>=1)])

for k,v in table.items():
    print(v+r" \\")


HNL_muonType_mHNL10p0_pl10000
HNL_muonType_mHNL10p0_pl1000
HNL_muonType_mHNL10p0_pl100
HNL_muonType_mHNL1p0_pl1000
HNL_muonType_mHNL1p0_pl100
HNL_muonType_mHNL1p0_pl10
HNL_muonType_mHNL2p0_pl1000
HNL_muonType_mHNL2p0_pl100
HNL_muonType_mHNL2p0_pl10
HNL_muonType_mHNL4p0_pl1000
HNL_muonType_mHNL4p0_pl100
HNL_muonType_mHNL4p0_pl10


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:208: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:216: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:224: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:233: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:239: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:247: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:254: RuntimeWarning: invalid value encountered in double_scalars


HNL_muonType_mHNL7p0_pl10000
HNL_muonType_mHNL7p0_pl1000
HNL_electronType_mHNL7p0_pl100


/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:167: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:174: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:180: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:191: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:201: RuntimeWarning: invalid value encountered in double_scalars


Acceptance  &  2.832 &  2.832 &  5.356 &  5.356 &  0.022 &  0.022 &  3.000 &  3.000 &  5.594 &  5.594 &  0.019 &  0.019 &  4.726 &  4.726 &  2.682 &  2.682 &  0.001 &  0.001 &  6.199 &  6.199 &  0.556 &  0.556 &  0.000 &  0.000 &  2.232 &  2.232 &  6.225 &  6.225 &  0.087 &  0.087 \\
single lepton trigger &  56.08 &  56.08 &  67.08 &  67.08 &  32.88 &  32.88 &  53.56 &  53.56 &  67.28 &  67.28 &  41.94 &  41.94 &  56.86 &  56.86 &  69.36 &  69.36 &  33.85 &  33.85 &  61.40 &  61.40 &  62.89 &  62.89 &  24.72 &  24.72 &  53.37 &  53.37 &  64.97 &  64.97 &   0.00 &   0.00 \\
$N_{lepton} > 0$  &  91.69 &  51.42 &  91.74 &  61.54 &  92.62 &  30.45 &  92.12 &  49.34 &  92.19 &  62.03 &  86.61 &  36.32 &  91.94 &  52.28 &  92.78 &  64.35 & 100.00 &  33.85 &  92.03 &  56.50 &  90.08 &  56.65 & 100.00 &  24.72 &  91.71 &  48.95 &  92.07 &  59.82 &    nan &   0.00 \\
MET cut  &  68.89 &  35.42 &  71.01 &  43.70 &  97.27 &  29.62 &  66.75 &  32.94 &  70.62 &  43.80 &  96.83 &  35.17 &  67.41 &  